In [1]:
import numpy as np
import pandas as pd
import random
import operator
import pandas as pd
import matplotlib.pyplot as plt

# Reading and Preprocessing Data

In [2]:
def read_excel(data):
    df = pd.read_excel('Data Prep.xlsx', index_col=0, sheet_name=data)
    return df

def read_excel_revisi(data):
    df = pd.read_excel('Data Prep Revisi.xlsx', index_col=0, sheet_name=data)
    return df



def mirroring(df): 
    df_c = df.copy()
    df = df.fillna(0)
    df = df + df.T
    df[df_c.isnull() & (df == 0)] = None
    return df

def set_same_zero(df):
    df_column = df.columns.values
    df_column = set(df_column)
    df_index = df.index.values
    df_index = set(df_index)
    same_port = df_index & df_column

    # making same port data into 0 
    for i in same_port:
        df[i][i]=0
    return df


In [3]:
alpha_up = read_excel('alpha_up')

GAMMA_TL = read_excel('GAMMA_TL')

w = read_excel('w')
w = mirroring(w)
w = set_same_zero(w)

TL_char = read_excel_revisi('TL_char')
PL_char = read_excel_revisi('PL_char')
PR_char = read_excel_revisi('PR_char')

# TL_char = read_excel_revisi('TL_char')
# PL_char = read_excel_revisi('PL_char')
# PR_char = read_excel_revisi('PR_char')

beta_up = read_excel('beta_up')

gamma_uu = read_excel('gamma_uu')
gamma_uu = mirroring(gamma_uu)
gamma_uu = set_same_zero(gamma_uu)


SIGMA_PL = read_excel('SIGMA_PL')



sigma_pr = read_excel('sigma_pr')

PSI_PR = read_excel('PSI_PR')

z_pr = read_excel('z_pr')

special_PR = pd.read_excel('Data Prep Revisi.xlsx', sheet_name='special_PR')

port_item = pd.read_excel('data.xlsx',index_col=0 , sheet_name='Barang')

biaya_jarak_teus = read_excel_revisi('Biaya_Jarak_Teus')

wave_status = pd.read_excel('Data Prep Revisi.xlsx', sheet_name='wave_status')

ports = pd.read_excel('Data Prep Revisi.xlsx', sheet_name='ports')

In [4]:
R_list = ports[ports['port_type'] == 'R']['port'].to_list()
P_list = ports[ports['port_type'] == 'P']['port'].to_list()
PL_P_list = ['Banda Neira','Dobo','Ambon','Tual','Saumlaki']
PL_non_P_list = list(set(P_list)-set(PL_P_list))

In [5]:
special_P_port = special_PR.columns.to_list()
wave_status['PR_availability'] = wave_status['wave_h']<2

In [6]:
TL1_data = pd.read_excel('Data Ship.xlsx', sheet_name='TL1')
TL2_data = pd.read_excel('Data Ship.xlsx', sheet_name='TL2')
PL1_data = pd.read_excel('Data Ship.xlsx', sheet_name='PL1')
PR_Ambon_data = pd.read_excel('Data Ship.xlsx', sheet_name='PR_Ambon')
PR_Saumlaki_data = pd.read_excel('Data Ship.xlsx', sheet_name='PR_Saumlaki')
PR_Tual_data = pd.read_excel('Data Ship.xlsx', sheet_name='PR_Tual')


#### Filling Data with Dummy Data

In [7]:
def fill_mean(df):
    df = df.fillna(df.mean().mean())
    return df

w = fill_mean(w)
SIGMA_PL = fill_mean(SIGMA_PL)
sigma_pr = fill_mean(sigma_pr)
z_pr = fill_mean(z_pr)
PSI_PR = fill_mean(PSI_PR)
PR_char = fill_mean(PR_char)
biaya_jarak_teus = fill_mean(biaya_jarak_teus)

# Data Dummy

In [8]:
port_item  = pd.read_csv('barang_dummy.csv',index_col=0)

# Creating Objects and Function

In [9]:
def quickport(port_name):
    name = ports[ports['port'] == port_name]['port'].values[0]
    port_type = ports[ports['port'] == port_name]['port_type'].values[0]
    return Port(name,port_type)

In [10]:
def chooseport(port_name):
    port_name = port_name.rstrip()
    port_name = port_name.lstrip()
    return ports[ports['port'] == port_name]['port_object'].iloc[0]

In [11]:
def port_items(port):
    global port_item
    item_list = port_item[port_item['Pelabuhan'] == port]
    return item_list

In [12]:
def find_distance(port_a,port_b):
    distance = np.random.rand()*100 #TODO: FIND DISTANCE
    return distance

In [13]:
def choose_char(ship_type):
    global TL_char, PL_char, PR_char
    if ship_type=='TL':
        ship_char = TL_char
    elif ship_type=='PL':
        ship_char = PL_char
    elif ship_type=='PR':
        ship_char = PR_char
    return ship_char

In [14]:
def per_item_revenue(data):
    distance = find_distance(data['Pelabuhan'],data['Tujuan Pelabuhan'])
    Bobot = data['Bobot']
    BJT = biaya_jarak_teus[data['Pelabuhan']][data['Tujuan Pelabuhan']]
    income = distance*Bobot*BJT
    return income

In [15]:
def port_revenue(storage_df):
    total_income = 0
    for i in storage_df.index:
        income = per_item_revenue(storage_df.loc[i])
        total_income += income
    return total_income

In [16]:
def item_left(df):
    return int(df['Bobot'].sum())

In [17]:
def find_movable_port(ship_type,port_name):
    if ship_type == 'TL':
        flag = False
    elif ship_type == 'PR':
        flag = True
    else:
        added_route = []
        return added_route
    r_list = special_PR[port_name].dropna().to_list()
    route_list = wave_status[wave_status['Port'].isin(r_list)][wave_status['PR_availability'] == flag]['Port'].to_list()
    added_route = []
    for i in route_list:
        added_route.append(chooseport(i))
    return added_route

In [18]:
def all_item(available_ship):
    itemdf = pd.DataFrame(columns = ['Pelabuhan','Bobot','Tujuan Pelabuhan'])
    itemdf.index.name = 'code barang'
    for i in ports['port_object']:
        itemdf = itemdf.append(i.demand)
    for i in available_ship:
        itemdf = itemdf.append(i.storage)
    return itemdf

In [19]:
def create_route(route_list):
    route = []
    for i in route_list:
        route.append(chooseport(i))
    return route
    

In [20]:
class Port:
    def __init__(self, name,port_type):
        self.name = name
        self.port_type = port_type
        self.demand = port_items(name)
        
    
    def __repr__(self):
        if self.port_type == 'U':
            port_type_name = 'Utama'
        elif self.port_type == 'P':
            port_type_name = 'Pengumpul'
        elif self.port_type == 'R':
            port_type_name = 'Pengumpan'
        else:
            port_type_name = 'NOT DEFINED'

        return port_type_name + ":" + self.name
    

In [21]:
def get_route(df):
    return df['Route'].to_list()

In [22]:
class Ship:
    def __init__(self, ship_type, route=[], name='No Name'):
        self.name = name
        self.ship_type = ship_type
        self.route_n = 0
        self.revenue = 0
        self.route = route
        self.current_port = route[0]
        self.port_cycle = route
        self.flag = False

        storage_df = pd.DataFrame(columns = ['Pelabuhan','Bobot','Tujuan Pelabuhan'])
        storage_df.index.name = 'code barang'
        self.storage = storage_df
        self.storage_volume = storage_df['Bobot'].sum()
        
        if ship_type == 'TL':
            portcycle_name =[]
            for i in self.port_cycle:
                portcycle_name.append(i.name)
            P_port_PR_transitport =  list(set(portcycle_name).intersection(set(special_PR)))
            P_port_non_PR_transitport = [x for x in special_PR if x not in P_port_PR_transitport]
            
            self.PR_transitport = []
            self.non_PR_transitport = []

            for i in P_port_PR_transitport:
                self.PR_transitport.extend(special_PR[i].dropna().to_list())
            for i in P_port_non_PR_transitport:
                self.non_PR_transitport.extend(special_PR[i].dropna().to_list())

    def update_storage_volume(self):
        self.storage_volume = self.storage['Bobot'].sum()
    def cycle_list(self):
        self.port_cycle = list(self.port_cycle[1:])+[self.port_cycle[0]]
    def move_port(self):
        if self.ship_type == 'TL':
            if self.current_port.name in special_P_port:
                if self.flag == False:
                    added_port = find_movable_port(self.ship_type,self.current_port.name)
                    self.cycle_list()
                    self.port_cycle = added_port+self.port_cycle
                    self.flag =True
                    print(self.port_cycle)
                    
                elif self.flag == True:
                    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% HERE %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
                    self.port_cycle = [x for x in self.port_cycle if x not in create_route(R_list)]
                    print(self.port_cycle)
                    self.flag=False

        elif self.ship_type == 'PR':
            if self.current_port.name in special_P_port:
                moveable_port = find_movable_port(self.ship_type,self.current_port.name) 
                self.port_cycle = [x for x in self.port_cycle if x in moveable_port ]
                
        self.cycle_list()
        self.current_port = self.port_cycle[0]
    def drop_on_port(self,port_destination):
        port_item = self.current_port.demand[self.current_port.demand['Tujuan Pelabuhan'] == port_destination]
        self.current_port.demand = pd.concat([self.current_port.demand, port_item]).drop_duplicates(keep=False)
        self.storage = self.storage.append(port_item)
        self.update_storage_volume()

        
    def move_storage(self):
        
        ship_char = choose_char(self.ship_type)

        # ITEM DROP OFF
        ship_item = self.storage[self.storage['Tujuan Pelabuhan'] == self.current_port.name]
        self.storage = pd.concat([self.storage, ship_item]).drop_duplicates(keep=False)
        self.update_storage_volume()
        
        
        special_ship_item = pd.DataFrame(columns=self.storage.columns)
        if self.current_port.name in ['Tanjung Perak','Makassar']:
            added_port = []
            portcycle_name =[]
            for i in self.port_cycle:
                portcycle_name.append(i.name)
            special_p_port = list(set(portcycle_name).intersection(set(special_PR)))
            for i in special_p_port:
                added_port.append(find_movable_port(self.ship_type,i))
            for i in set(self.storage['Tujuan Pelabuhan'].to_list()):
                if i not in self.route+added_port:
                    special_ship_item = special_ship_item.append(self.storage[self.storage['Tujuan Pelabuhan'] == i])
            if self.ship_type == 'TL':
                for i in self.non_PR_transitport:
                    special_ship_item = special_ship_item.append(self.storage[self.storage['Tujuan Pelabuhan'] == i])


            
        if self.current_port.name in special_PR:
            if self.ship_type == 'TL':
                pass
            elif self.ship_type == 'PL':
                r_list = special_PR[self.current_port.name].dropna().to_list()
                for i in r_list:
                    special_ship_item = special_ship_item.append(self.storage[self.storage['Tujuan Pelabuhan'] == i])

        
        self.storage = pd.concat([self.storage, special_ship_item]).drop_duplicates(keep=False)
        self.update_storage_volume()

        
        ship_item = ship_item.append(special_ship_item)
        ship_item = ship_item.drop_duplicates()
        #CALCULATING SHIP REVENUE
        self.revenue += port_revenue(ship_item)
        
        #CALCULATING SHIP COST        
        total_cost_bongkar = 0
        total_cost_storage = 0
        total_cost_travel_time = 0
        cost_bongkar_time = 0
        for i in list(ship_item.index):
            item = ship_item.loc[i]
            Bobot = item['Bobot']
            bm_time = ship_char[self.current_port.name ]['bm_time']
            C_bm = ship_char[self.current_port.name ]['C_bm']
            inventory_cost = ship_char[self.current_port.name ]['inventory_cost']
            bm_time = ship_char[self.current_port.name ]['bm_time']
            avg_docking_time = ship_char[self.current_port.name ]['avg_docking_time']
            distance = find_distance(item['Pelabuhan'],self.current_port.name )
            V = ship_char['ship_char']['V']
            VC = ship_char['ship_char']['VC']
            port_storage_time = ship_char[self.current_port.name ]['port_storage_time']
            C_storage = ship_char[self.current_port.name ]['C_storage']

            total_cost_bongkar += Bobot*bm_time*C_bm
            total_cost_storage += Bobot * port_storage_time * C_storage
            total_cost_travel_time += inventory_cost*bm_time*(avg_docking_time*(distance/V))
            cost_bongkar_time += inventory_cost * (Bobot/C_bm)

        travel_cost = total_cost_bongkar+ total_cost_storage+total_cost_travel_time+cost_bongkar_time
        self.revenue -= travel_cost
        
        print('port_revenue : '+str(port_revenue(ship_item))+ ' travel_cost : ' + str(travel_cost))
        
        special_ship_item['Pelabuhan'] = self.current_port.name
        self.current_port.demand = self.current_port.demand.append(special_ship_item)




        
        # ITEM Pick Up
        for i in self.port_cycle:
            self.drop_on_port(i.name)
        if self.ship_type == 'TL':
            if self.current_port.name not in ['Makassar','Tanjung Perak']:
                self.drop_on_port('Tanjung Priok')
            if self.current_port.name in ['Makassar','Tanjung Perak']:
                for i in self.PR_transitport:
                    self.drop_on_port(i)
            if self.current_port.name not in R_list:
                for i in R_list:
                    self.drop_on_port(i)
            if self.current_port.name in R_list:
                self.current_port.demand = pd.concat([self.current_port.demand, self.current_port.demand]).drop_duplicates(keep=False)
                self.storage = self.storage.append(self.current_port.demand)
                self.update_storage_volume()
        if self.ship_type == 'PL':
            if self.current_port.name in ['Makassar','Tanjung Perak']:
                self.drop_on_port('Tanjung Priok')
            if self.current_port.name == 'Tanjung Priok':
                for i in R_list+PL_non_P_list:
                    self.drop_on_port(i)
        if self.ship_type == 'PR':
            if self.current_port.name not in special_P_port:
                self.current_port.demand = pd.concat([self.current_port.demand, self.current_port.demand]).drop_duplicates(keep=False)
                self.storage = self.storage.append(self.current_port.demand)
                self.update_storage_volume()
            
            
    
      
    def __repr__(self):
        return self.ship_type +': '+ self.ship_type


In [23]:
ports['port_object'] = ports['port'].apply(lambda x: quickport(x))

In [24]:
route_list = ['Tanjung Perak','Saumlaki','Tanjung Priok','Dobo','Namlea']
route = create_route(route_list)

In [25]:
route_list_TL1 = get_route(TL1_data)
route_TL1 = create_route(route_list_TL1)

route_list_TL2 = get_route(TL2_data)
route_TL2 = create_route(route_list_TL2)

route_list_PL1 = get_route(PL1_data)
route_PL1 = create_route(route_list_PL1)

route_list_PR_Ambon = get_route(PR_Ambon_data)
route_PR_Ambon = create_route(route_list_PR_Ambon)

route_list_PR_Tual = get_route(PR_Tual_data)
route_PR_Tual = create_route(route_list_PR_Tual)

route_list_PR_Saumlaki = get_route(PR_Saumlaki_data)
route_PR_Saumlaki = create_route(route_list_PR_Saumlaki)


In [26]:
TL_tester = Ship('TL',route)
TL1 = Ship('TL',route_TL1)
TL2 = Ship('TL',route_TL2)
PL1 = Ship('PL',route_PL1)
PR_Ambon = Ship('PR',route_PR_Ambon)
PR_Tual = Ship('PR',route_PR_Tual)
PR_Saumlaki = Ship('PR',route_PR_Saumlaki)



In [27]:
available_ship = [TL1,TL2,PR_Ambon,PR_Tual,PR_Saumlaki]

In [28]:
def port_sequence(ship):
    print('')
    print('')
    print('')
    print('')
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
    print('ship_type: '+ship.ship_type)
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
    print('BEFORE')
    print('current port:' + ship.current_port.name)
    print('')
    print('PORT DEMAND:')
    print(ship.current_port.demand)
    print('')
    print('SHIP STORAGE:')
    print(ship.storage)
    ship.move_storage()
    print('oOoOoOoOoOoOoOoOoOoOoOoO')
    print('AFTER')
    print('current port:' + ship.current_port.name)
    print('')
    print('PORT DEMAND:')
    print(ship.current_port.demand)
    print('')
    print('SHIP STORAGE:')
    print(ship.storage)
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
    ship.move_port()
    ship.route_n += 1


In [29]:
new_port_item = all_item(available_ship)
remaining = item_left(new_port_item)
i=0
while remaining>0:
    i +=1
#     port_sequence(TL_tester)
    port_sequence(TL1)
    port_sequence(TL2)
    port_sequence(PL1)
    port_sequence(PR_Ambon)
    port_sequence(PR_Tual)
    port_sequence(PR_Saumlaki)
    new_port_item = all_item(available_ship)
    remaining = item_left(new_port_item)
    print('************************************************')
    print('remaining: ' + str(remaining))
    print('iteration: ' + str(i))
    print('************************************************')

    if i>=800:
        break







$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
ship_type: TL
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
BEFORE
current port:Tanjung Perak

PORT DEMAND:
          Pelabuhan  Bobot    Tujuan Pelabuhan
A2    Tanjung Perak     89              Romang
A93   Tanjung Perak     41                Nila
A184  Tanjung Perak     33  Sofyanin/ Rumayaan
A275  Tanjung Perak     18           Waemulang

SHIP STORAGE:
Empty DataFrame
Columns: [Pelabuhan, Bobot, Tujuan Pelabuhan]
Index: []
port_revenue : 0 travel_cost : 0
oOoOoOoOoOoOoOoOoOoOoOoO

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.



AFTER
current port:Tanjung Perak

PORT DEMAND:
Empty DataFrame
Columns: [Pelabuhan, Bobot, Tujuan Pelabuhan]
Index: []

SHIP STORAGE:
          Pelabuhan Bobot    Tujuan Pelabuhan
A93   Tanjung Perak    41                Nila
A2    Tanjung Perak    89              Romang
A275  Tanjung Perak    18           Waemulang
A184  Tanjung Perak    33  Sofyanin/ Rumayaan
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$




$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
ship_type: TL
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
BEFORE
current port:Makassar

PORT DEMAND:
     Pelabuhan  Bobot   Tujuan Pelabuhan
A3    Makassar     84           P. Luang
A94   Makassar     14  Arwala/Sutilirang
A185  Makassar     93            Ambalau
A276  Makassar     31      Tanjung Priok

SHIP STORAGE:
Empty DataFrame
Columns: [Pelabuhan, Bobot, Tujuan Pelabuhan]
Index: []
port_revenue : 0 travel_cost : 0
oOoOoOoOoOoOoOoOoOoOoOoO
AFTER
current port:Makassar

PORT DEMAND:
     Pelabuhan Bobot Tujuan Pelabuhan
A276  Makassar    31    Tanjung Priok

SHIP S

************************************************
remaining: 17713
iteration: 2
************************************************




$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
ship_type: TL
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
BEFORE
current port:Wulur

PORT DEMAND:
     Pelabuhan  Bobot Tujuan Pelabuhan
A72      Wulur     66           Manipa
A163     Wulur     46         Werinama
A254     Wulur     59         Werinama
A345     Wulur     54     Lelam Kojabi

SHIP STORAGE:
          Pelabuhan Bobot    Tujuan Pelabuhan
A93   Tanjung Perak    41                Nila
A2    Tanjung Perak    89              Romang
A275  Tanjung Perak    18           Waemulang
A184  Tanjung Perak    33  Sofyanin/ Rumayaan
A98        Saumlaki     9               Geser
A280       Saumlaki    12            Kelimuri
A189       Saumlaki    97               Fadol
port_revenue : 0 travel_cost : 0
oOoOoOoOoOoOoOoOoOoOoOoO
AFTER
current port:Wulur

PORT DEMAND:
Empty DataFrame
Columns: [Pelabuhan, Bobot, Tujuan Pelabuhan]
Index: []



oOoOoOoOoOoOoOoOoOoOoOoO
AFTER
current port:Rumean

PORT DEMAND:
Empty DataFrame
Columns: [Pelabuhan, Bobot, Tujuan Pelabuhan]
Index: []

SHIP STORAGE:
     Pelabuhan Bobot Tujuan Pelabuhan
A7    Saumlaki    42             Nila
A148    Rumean    51           Kisar1
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
************************************************
remaining: 16303
iteration: 4
************************************************




$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
ship_type: TL
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
BEFORE
current port:Wunlah

PORT DEMAND:
     Pelabuhan  Bobot Tujuan Pelabuhan
A58     Wunlah     55      Tutukembong
A149    Wunlah     39             Bula
A240    Wunlah     36             Tifu
A331    Wunlah     53            Larat

SHIP STORAGE:
              Pelabuhan Bobot    Tujuan Pelabuhan
A93       Tanjung Perak    41                Nila
A2        Tanjung Perak    89              Romang
A275      Tanjung Perak    18           Waemulang
A184      Tanjung Perak    33  Sofyani

...

A188  Tanjung Perak    29          Lewa / Dai
port_revenue : 0 travel_cost : 0
oOoOoOoOoOoOoOoOoOoOoOoO
AFTER
current port:Moa

PORT DEMAND:
Empty DataFrame
Columns: [Pelabuhan, Bobot, Tujuan Pelabuhan]
Index: []

SHIP STORAGE:
          Pelabuhan Bobot    Tujuan Pelabuhan
A151  Tanjung Perak    57                Tepa
A93   Tanjung Perak    41                Nila
A1    Tanjung Perak    46           Waemulang
A275  Tanjung Perak    18           Waemulang
A279  Tanjung Perak    23           Waemulang
A274  Tanjung Perak    42               Wahai
A280  Tanjung Perak    12            Kelimuri
A98   Tanjung Perak     9               Geser
A183  Tanjung Perak     7            Werinama
A184  Tanjung Perak    33  Sofyanin/ Rumayaan
A188  Tanjung Perak    29          Lewa / Dai
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$




$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
ship_type: TL
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
BEFORE
current port:Moa

PORT DEMAND:
Empty DataFrame
Columns: [Pelabuhan, Bobot, Tujuan Pelabuhan]
Ind

************************************************
remaining: 7356
iteration: 214
************************************************




$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
ship_type: TL
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
BEFORE
current port:Moa

PORT DEMAND:
Empty DataFrame
Columns: [Pelabuhan, Bobot, Tujuan Pelabuhan]
Index: []

SHIP STORAGE:
          Pelabuhan Bobot    Tujuan Pelabuhan
A151  Tanjung Perak    57                Tepa
A93   Tanjung Perak    41                Nila
A1    Tanjung Perak    46           Waemulang
A275  Tanjung Perak    18           Waemulang
A279  Tanjung Perak    23           Waemulang
A274  Tanjung Perak    42               Wahai
A280  Tanjung Perak    12            Kelimuri
A98   Tanjung Perak     9               Geser
A183  Tanjung Perak     7            Werinama
A184  Tanjung Perak    33  Sofyanin/ Rumayaan
A188  Tanjung Perak    29          Lewa / Dai
port_revenue : 0 travel_cost : 0
oOoOoOoOoOoOoOoOoOoOoOoO
AFTER
current port:Moa

PORT DEMAND:
Empty DataFrame

************************************************
remaining: 7356
iteration: 216
************************************************




$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
ship_type: TL
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
BEFORE
current port:Moa

PORT DEMAND:
Empty DataFrame
Columns: [Pelabuhan, Bobot, Tujuan Pelabuhan]
Index: []

SHIP STORAGE:
          Pelabuhan Bobot    Tujuan Pelabuhan
A151  Tanjung Perak    57                Tepa
A93   Tanjung Perak    41                Nila
A1    Tanjung Perak    46           Waemulang
A275  Tanjung Perak    18           Waemulang
A279  Tanjung Perak    23           Waemulang
A274  Tanjung Perak    42               Wahai
A280  Tanjung Perak    12            Kelimuri
A98   Tanjung Perak     9               Geser
A183  Tanjung Perak     7            Werinama
A184  Tanjung Perak    33  Sofyanin/ Rumayaan
A188  Tanjung Perak    29          Lewa / Dai
port_revenue : 0 travel_cost : 0
oOoOoOoOoOoOoOoOoOoOoOoO
AFTER
current port:Moa

PORT DEMAND:
Empty DataFrame

port_revenue : 0 travel_cost : 0
oOoOoOoOoOoOoOoOoOoOoOoO
AFTER
current port:Moa

PORT DEMAND:
Empty DataFrame
Columns: [Pelabuhan, Bobot, Tujuan Pelabuhan]
Index: []

SHIP STORAGE:
          Pelabuhan Bobot    Tujuan Pelabuhan
A151  Tanjung Perak    57                Tepa
A93   Tanjung Perak    41                Nila
A1    Tanjung Perak    46           Waemulang
A275  Tanjung Perak    18           Waemulang
A279  Tanjung Perak    23           Waemulang
A274  Tanjung Perak    42               Wahai
A280  Tanjung Perak    12            Kelimuri
A98   Tanjung Perak     9               Geser
A183  Tanjung Perak     7            Werinama
A184  Tanjung Perak    33  Sofyanin/ Rumayaan
A188  Tanjung Perak    29          Lewa / Dai
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$




$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
ship_type: TL
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
BEFORE
current port:Moa

PORT DEMAND:
Empty DataFrame
Columns: [Pelabuhan, Bobot, Tujuan Pelabuhan]
Index: []

SHIP STORAGE:
   Pelabuhan Bobot Tujua

KeyboardInterrupt: 

In [ ]:
port_item.loc['A97']

In [ ]:
TL1.port_cycle

In [ ]:
'Wamsisi' in TL1.PR_transitport

In [ ]:
'Sofyanin/Rumayaan' in TL1.PR_transitport

In [ ]:
TL1.PR_transitport

In [ ]:
portcycle_name =[]
for i in TL1.port_cycle:
    portcycle_name.append(i.name)
TL1.PR_transitport =  list(set(portcycle_name).intersection(set(special_PR)))
TL1.non_PR_transitport = [x for x in special_PR if x not in TL1.PR_transitport]


In [ ]:
TL1.PR_transitport 

In [ ]:
a = ['a','b','c']
b = ['d','e']
a = b+a
a

In [ ]:
biaya_jarak_teus['Saumlaki']['Fadol']